In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Sergipe - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego,Sergipe - Consumo de Cimento (t)
0,2003-1,0.669217,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.339447,152.791400,2.159732e+07,1.260978e+06,9.960678,2.029642e+07,8.389943,23.616
1,2003-2,0.669542,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.341257,151.964054,2.162153e+07,1.262134e+06,9.961956,2.030238e+07,8.384067,21.357
2,2003-3,0.669868,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.343016,154.467758,2.164574e+07,1.263289e+06,9.963234,2.030833e+07,8.378190,20.059
3,2003-4,0.670193,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.344722,155.913400,2.166995e+07,1.264444e+06,9.964512,2.031429e+07,8.372313,19.920
4,2003-5,0.670519,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.346174,157.467159,2.169416e+07,1.265599e+06,9.965790,2.032025e+07,8.366437,19.962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.557749,169.752591,NaN,NaN,NaN,NaN,NaN,34.037
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.556659,169.621865,NaN,NaN,NaN,NaN,NaN,32.537
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.555237,169.855545,NaN,NaN,NaN,NaN,NaN,35.215
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.553445,170.188727,NaN,NaN,NaN,NaN,NaN,34.707


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
0,0.000000,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.006994,0.000000,0.533618,0.785466,0.000000,0.044357
1,0.010381,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.012115,0.000000,0.007166,0.543393,0.792310,0.021047,0.043343
2,0.020763,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.023890,0.021167,0.014333,0.553168,0.799153,0.042095,0.042329
3,0.031144,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.035313,0.033388,0.021499,0.562943,0.805997,0.063142,0.041315
4,0.041526,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.045034,0.046524,0.028665,0.572718,0.812840,0.084190,0.040302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.984075,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.778903,0.163281,0.967983,0.015526,0.105652,0.520107,0.937760
188,0.981800,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.777382,0.166261,0.965137,0.017744,0.109196,0.521093,0.936030
189,0.979525,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.777849,0.179065,0.962292,0.019962,0.112740,0.522080,0.934300
190,0.977250,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.782320,0.182794,0.959446,0.022180,0.116284,0.523066,0.932570


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      19.642
1      17.778
2      21.180
3      18.824
4      16.535
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Sergipe - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
0,0.000000,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.006994,0.000000,0.533618,0.785466,0.000000,0.044357
1,0.010381,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.012115,0.000000,0.007166,0.543393,0.792310,0.021047,0.043343
2,0.020763,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.023890,0.021167,0.014333,0.553168,0.799153,0.042095,0.042329
3,0.031144,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.035313,0.033388,0.021499,0.562943,0.805997,0.063142,0.041315
4,0.041526,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.045034,0.046524,0.028665,0.572718,0.812840,0.084190,0.040302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.987398,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.984297,0.486237,0.986615,0.195898,0.003620,0.482795,0.997065
158,0.988162,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.976346,0.468165,0.987832,0.184141,0.007240,0.486671,0.994130
159,0.988925,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.968320,0.447949,0.989049,0.172385,0.010860,0.490546,0.991194
160,0.989689,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.960498,0.423404,0.990265,0.160628,0.014480,0.494422,0.988259


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      19.642
1      17.778
2      21.180
3      18.824
4      16.535
        ...  
157    37.579
158    48.016
159    38.263
160    40.413
161    34.053
Name: Sergipe - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Sergipe - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Sergipe - value,Sergipe - Produção de Cimento (t),Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - Desemprego
126,0.707912,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.703747,1.000000,0.888454,0.569106,0.347678,0.831947,0.807761
127,0.709701,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.719291,0.997781,0.893743,0.554339,0.332523,0.819306,0.815945
128,0.711491,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.735075,0.986916,0.899032,0.539572,0.317368,0.806666,0.824129
129,0.713281,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.752417,0.976767,0.904320,0.524805,0.302213,0.794025,0.832313
130,0.715070,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.768589,0.966401,0.909609,0.510038,0.287057,0.781384,0.840497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.984075,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.778903,0.163281,0.967983,0.015526,0.105652,0.520107,0.937760
188,0.981800,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.777382,0.166261,0.965137,0.017744,0.109196,0.521093,0.936030
189,0.979525,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.777849,0.179065,0.962292,0.019962,0.112740,0.522080,0.934300
190,0.977250,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.782320,0.182794,0.959446,0.022180,0.116284,0.523066,0.932570


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3650033993, 38064029, 1614896854, 3946976089, 3620069980, 2892366589, 3242080579, 1090277615, 2698741670, 2006707507]


Step: 0 ___________________________________________
val_loss: 7.229344844818115
winner_seed: 3650033993


Step: 1 ___________________________________________
val_loss: 8.190119743347168


Step: 2 ___________________________________________
val_loss: 6.835001468658447
winner_seed: 1614896854


Step: 3 ___________________________________________
val_loss: 9.138724327087402


Step: 4 ___________________________________________
val_loss: 9.144423484802246


Step: 5 ___________________________________________
val_loss: 6.302597522735596
winner_seed: 2892366589


Step: 6 ___________________________________________
val_loss: 7.702000141143799


Step: 7 ___________________________________________
val_loss: 8.300325393676758


Step: 8 ___________________________________________
val_loss: 7.092090129852295


Step: 9 ___________________________________________
val_loss: 7.93523

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 110ms/step - loss: 1908.3591 - val_loss: 1727.0723
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 1638.8263 - val_loss: 1483.6539
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 1415.8982 - val_loss: 1289.3995
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 1229.0623 - val_loss: 1117.9353
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 1063.2188 - val_loss: 977.2167
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 933.8098 - val_loss: 863.7278
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 826.8874 - val_loss: 770.6886
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 739.7643 - val_loss: 690.0320
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 661.5105 - val_loss: 617.6711
Epoch 10/10000
4/4 [==============================] - 0s 8ms/s

4/4 [==============================] - 0s 7ms/step - loss: 95.3653 - val_loss: 100.4733
Epoch 81/10000
4/4 [==============================] - 0s 7ms/step - loss: 94.9801 - val_loss: 100.1812
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 94.6546 - val_loss: 99.9383
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 94.3388 - val_loss: 99.6352
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 94.0373 - val_loss: 99.4168
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.8216 - val_loss: 99.2267
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.6255 - val_loss: 99.0648
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.4454 - val_loss: 98.9173
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.2721 - val_loss: 98.7150
Epoch 89/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.1122 - val_loss: 9

Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.4066 - val_loss: 35.4383
Epoch 161/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.1384 - val_loss: 36.3605
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.7767 - val_loss: 36.5510
Epoch 163/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.7897 - val_loss: 34.9974
Epoch 164/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.8399 - val_loss: 35.0024
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.6520 - val_loss: 35.4813
Epoch 166/10000
4/4 [==============================] - 0s 8ms/step - loss: 16.4670 - val_loss: 34.9745
Epoch 167/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.3588 - val_loss: 35.2272
Epoch 168/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.3687 - val_loss: 35.9134
Epoch 169/10000
4/4 [==============================] - 0s 7ms/step - loss

4/4 [==============================] - 0s 7ms/step - loss: 14.1870 - val_loss: 31.3191
Epoch 240/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.5340 - val_loss: 33.4213
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.0690 - val_loss: 33.8022
Epoch 242/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.7739 - val_loss: 31.7728
Epoch 243/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.6030 - val_loss: 30.6068
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.8472 - val_loss: 33.3007
Epoch 245/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.6985 - val_loss: 31.9026
Epoch 246/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.4009 - val_loss: 31.1700
Epoch 247/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.1693 - val_loss: 29.4293
Epoch 248/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.7304 - val_

4/4 [==============================] - 0s 7ms/step - loss: 7.8755 - val_loss: 18.9263
Epoch 320/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5594 - val_loss: 18.1298
Epoch 321/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8221 - val_loss: 17.8906
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5652 - val_loss: 19.2528
Epoch 323/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6136 - val_loss: 19.4065
Epoch 324/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6958 - val_loss: 18.1850
Epoch 325/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6894 - val_loss: 17.1977
Epoch 326/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.9062 - val_loss: 18.0578
Epoch 327/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.8293 - val_loss: 19.0781
Epoch 328/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.3764 - val_loss: 19.4

...

Epoch 1994/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7897 - val_loss: 12.2098
Epoch 1995/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6714 - val_loss: 10.4652
Epoch 1996/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6197 - val_loss: 10.9145
Epoch 1997/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6810 - val_loss: 11.9801
Epoch 1998/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7120 - val_loss: 10.6909
Epoch 1999/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6826 - val_loss: 9.6004
Epoch 2000/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.5270 - val_loss: 10.8034
Epoch 2001/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.5318 - val_loss: 10.3527
Epoch 2002/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8491 - val_loss: 9.5211
Epoch 2003/10000
4/4 [==============================] - 0s 7ms/step - loss:

4/4 [==============================] - 0s 7ms/step - loss: 2.8283 - val_loss: 11.6350
Epoch 2074/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8752 - val_loss: 11.4402
Epoch 2075/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6235 - val_loss: 9.7193
Epoch 2076/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8106 - val_loss: 9.7809
Epoch 2077/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.5713 - val_loss: 10.8636
Epoch 2078/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7976 - val_loss: 10.5551
Epoch 2079/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.9332 - val_loss: 11.4140
Epoch 2080/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8734 - val_loss: 9.6107
Epoch 2081/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8663 - val_loss: 7.6132
Epoch 2082/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.9381 - val_loss:

Epoch 2153/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.3620 - val_loss: 9.2340
Epoch 2154/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.5282 - val_loss: 9.2484
Epoch 2155/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.4145 - val_loss: 10.7214
Epoch 2156/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.3663 - val_loss: 10.8074
Epoch 2157/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.2902 - val_loss: 9.3860
Epoch 2158/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.4628 - val_loss: 11.5732
Epoch 2159/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7814 - val_loss: 10.5818
Epoch 2160/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.2484 - val_loss: 8.7822
Epoch 2161/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8921 - val_loss: 11.3653
Epoch 2162/10000
4/4 [==============================] - 0s 7ms/step - loss: 2

4/4 [==============================] - 0s 8ms/step - loss: 2.0696 - val_loss: 10.8366
Epoch 2233/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.2597 - val_loss: 11.2158
Epoch 2234/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.2767 - val_loss: 11.0176
Epoch 2235/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.8804 - val_loss: 11.1182
Epoch 2236/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.6830 - val_loss: 11.2525
Epoch 2237/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7586 - val_loss: 9.5793
Epoch 2238/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.7309 - val_loss: 11.5630
Epoch 2239/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.3068 - val_loss: 11.3913
Epoch 2240/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.8757 - val_loss: 11.2073
Epoch 2241/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8827 - val_lo

Epoch 2312/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.0547 - val_loss: 10.0128
Epoch 2313/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6550 - val_loss: 9.7924
Epoch 2314/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6272 - val_loss: 9.9101
Epoch 2315/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.5785 - val_loss: 10.5291
Epoch 2316/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.5282 - val_loss: 9.6862
Epoch 2317/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6103 - val_loss: 10.0496
Epoch 2318/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8001 - val_loss: 11.8934
Epoch 2319/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8555 - val_loss: 11.3596
Epoch 2320/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.3733 - val_loss: 10.2742
Epoch 2321/10000
4/4 [==============================] - 0s 7ms/step - loss: 

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,39.676762,39.169033,38.832455,38.769085,38.751598,38.690933,38.651592,38.744236,38.685883,38.684654,38.699162,38.775036,38.584145,38.72258,38.815643,38.673199,38.580578,38.512646,38.661251,38.762028,38.641602,38.519054,38.554028,38.449154,38.521427,38.343845,38.390919,38.280758,38.358643,38.288792
Target,36.174,38.394,35.688,42.076,39.875,38.984,41.652,33.904,29.447,29.197,26.297,29.209,28.158,31.132,30.195,31.537,30.748,30.734,36.567,30.849,31.021,31.437,31.533,26.431,25.978,29.86,39.045,45.213,42.237,39.732
Error,3.502762,0.775032,3.144455,3.306915,1.123402,0.293068,3.000408,4.840237,9.238882,9.487654,12.402161,9.566036,10.426144,7.59058,8.620644,7.136198,7.832579,7.778646,2.09425,7.913027,7.620602,7.082054,7.021027,12.018154,12.543427,8.483845,0.654079,6.932243,3.878357,1.443207


In [38]:
display(mae)
display(mape)

6.2583356

0.20443323

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[466.13043]] - Target[420.89700000000005]| =  Error: [[45.23343]]; MAPE:[[0.10746912]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[463.4759]] - Target[370.34200000000004]| =  Error: [[93.13388]]; MAPE:[[0.25148073]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[230.18439]] - Target[222.065]| =  Error: [[8.119385]]; MAPE:[[0.0365631]]


[array([[45.23343]], dtype=float32),
 array([[93.13388]], dtype=float32),
 array([[8.119385]], dtype=float32)]

48.8289

0.13183765